参考了 DecryptLogin 案例中的 NeteaseSongListDownloader

两个 bug
1. 使用 getPlayListSongs 查看歌单列表的时候会卡着不动
2. 下载歌曲的时候间或有 bug，用了 try 简单跳过了

不知道这里 <https://yyrcd.com/n2s/> 用的是什么 API

In [ ]:
import os
import re
import click
import argparse
import prettytable
from contextlib import closing
from DecryptLogin import login
from DecryptLogin.modules.core.music163 import Cracker

In [ ]:
username = ''
password = ''

In [ ]:
def login163(username, password):
    lg = login.Login()
    infos_return, session = lg.music163(username, password)
    return infos_return.get('userid'), session

userid, session = login163(username, password)

In [ ]:
csrf = re.findall('__csrf=(.*?) for', str(session.cookies))[0]
cracker = Cracker()
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'
}

In [ ]:
# csrf, session.cookies

In [ ]:
import requests

'''获得所有歌单'''
def getPlayLists():
  playlist_url = 'https://music.163.com/weapi/user/playlist?csrf_token='
  playlists = []
  offset = 0
  while True:
    data = {
          "offset": offset,
          "uid": userid,
          "limit": 50,
          "csrf_token": csrf
        }
    res = session.post(playlist_url+csrf, headers=headers, data=cracker.get(data), timeout=5)

    playlists += res.json()['playlist']
    offset += 1
    if not res.json()['more'] == 'false':
      break
  all_playlists = {}
  for item in playlists:
    name = item.get('name')
    track_count = item.get('trackCount')
    play_count = item.get('playCount')
    play_id = item.get('id')
    if item.get('creator').get('userId') == userid:
      attr = '我创建的歌单'
    else:
      attr = '我收藏的歌单'
    all_playlists[str(play_id)] = [name, track_count, play_count, attr]
  return all_playlists


In [ ]:
all_playlists = getPlayLists()
tb = prettytable.PrettyTable()
tb.field_names = ['歌单ID', '歌单名', '歌曲数量', '播放次数', '歌单属性']
for key, value in all_playlists.items():
    tb.add_row([key]+value)
print('您创建/收藏的所有歌单如下:')
print(tb)

In [ ]:
'''获得某歌单的所有歌曲信息'''
import requests

def getPlayListSongs( playlist_id, num_songs):
    detail_url = 'https://music.163.com/weapi/v6/playlist/detail?csrf_token='
    offset = 0
    song_infos = {}
    while True:
        data = {
            'id': playlist_id,
            'offset': offset,
            'total': True,
            'limit': 1000,
            'n': 1000,
            'csrf_token': csrf
        }
        try:
            res = session.post(detail_url+csrf, headers=headers, data=cracker.get(data), timeout=5)
        except requests.exceptions.ConnectionError as e:
            print(str(e))
            return e

        tracks = res.json()['playlist']['tracks']
        for track in tracks:
            name = track.get('name')
            songid = track.get('id')
            artists = ','.join([i.get('name') for i in track.get('ar')])
            brs = [track.get('h')] + [track.get('m')] + [track.get('l')]
            song_infos[songid] = [name, artists, brs]
        offset += 1
        if len(list(song_infos.keys())) >= num_songs:
            break
    return song_infos

In [ ]:
# playlist_id, (playlist_name, playlist_len, play_times, playlist_type) = list(all_playlists.items())[0]
# song_infos = getPlayListSongs(playlist_id, playlist_len)

In [ ]:
# 导出所有歌单信息
# def output_playlists():
import json
from tqdm import tqdm
import time
import signal

playlist_info_list = []
for playlist_id, (playlist_name, playlist_len, play_times, playlist_type) in tqdm(list(all_playlists.items())[35:]):
    # time.sleep(3)
    try:
        song_infos = getPlayListSongs(playlist_id, playlist_len)
    except Exception as e:
        print("Error!!", playlist_name, playlist_id)
    playlist_i = {
        "id": playlist_id, 
        "name": playlist_name, 
        "len": playlist_len, 
        "times": play_times, 
        "type": playlist_type,
        "songs": song_infos
    }
    playlist_info_list.append(playlist_i)

In [ ]:
list(all_playlists.items())[35]

In [ ]:
playlist_info_list_ = [json.dumps(i, ensure_ascii=False) for i in playlist_info_list]
fout = open('163music-mylist-2.json', 'w')
for i in playlist_info_list_:
    fout.write(i+'\n')
# fout.writelines(playlist_info_list_)
# output_playlists()

In [ ]:
# import json
# json.dumps(playlist_info_list[0]['songs'], ensure_ascii=False)

In [ ]:
# playlist_id = '75649888'
# song_infos = getPlayListSongs(playlist_id, all_playlists[playlist_id][1])

# 分离关联
# playlist_id = '6764091275'
# playlist_len = 20
playlist_id = '5454637196'
playlist_len = 20
song_infos = getPlayListSongs(playlist_id, playlist_len)
tb = prettytable.PrettyTable()
tb.field_names = ['歌曲ID', '歌曲名', '歌手']
for key, value in song_infos.items():
    tb.add_row([key]+value[:-1])
print('您输入的歌单ID为<%s>, 该歌单的所有歌曲信息如下:' % playlist_id)
print(tb)

In [ ]:
'''下载某首歌曲'''
def downloadSong( songid, songname, brs, savepath='.'):
    play_url = 'http://music.163.com/weapi/song/enhance/player/url?csrf_token='
    print('正在下载 ——> %s' % songname)
    for br in brs:
        data = {
            'ids': [songid],
            'br': br.get('br'),
            'csrf_token': csrf
        }
        res = session.post(play_url+csrf, headers=headers, data=cracker.get(data))
        if res.json()['code'] == 200:
            download_url = res.json()['data'][0].get('url', '')
            if download_url:
                break
    with closing(session.get(download_url, headers=headers, stream=True, verify=False)) as res:
        total_size = int(res.headers['content-length'])
        if res.status_code == 200:
            label = '[FileSize]:%0.2f MB' % (total_size/(1024*1024))
            with click.progressbar(length=total_size, label=label) as progressbar:
                with open(os.path.join(savepath, songname+'.'+download_url.split('.')[-1]), "wb") as f:
                    for chunk in res.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            progressbar.update(1024)

In [ ]:
# savepath = all_playlists[playlist_id][0]
import time
savepath = "『粤语』中山大学学子私藏港乐"
if not os.path.exists(savepath):
    os.mkdir(savepath)
for key, value in song_infos.items():
    try:
        downloadSong(key, value[0], value[-1], savepath)
        # time.sleep(3)
    except Exception as e:
        print("Error!!", str(e))
print('歌单ID为<%s>中的所有歌曲下载完成, 保存在 ——> %s' % (playlist_id, savepath))

In [ ]:
list(song_infos.items())[0]

In [ ]:
len(list(song_infos.items()))